In [1]:
%matplotlib inline
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('float_format', '{:f}'.format)

def plotPlayer(id):
    player = id
    fig = plt.figure(figsize=(10, 6))
    ax = fig.gca()
    df[df['id'] == player][['wrating', 'age']].set_index('age').sort_index().plot(ax=ax)
    df[df['id'] == player][['rating', 'age']].set_index('age').sort_index().plot(ax=ax)
    return (df[df['id'] == player][['wrating', 'tour', 'age', 'team', 'start']].set_index('age').sort_index())

In [2]:
def load_csv(system):
    df = pd.read_csv(system + '.csv', encoding="utf-8")
    df.start = pd.to_datetime(df.start, format='%d/%m/%Y')
    df.end = pd.to_datetime(df.end, format='%d/%m/%Y')
    dob = pd.read_csv('dob.csv', encoding="utf-8")
    dob.id = dob.id.str.replace(r'\W+', '').str.lower()
    return df, dob

def cumulativeRating(diff, ratingName, df):
    cumsums = df[['id', 'start', diff]].groupby(['id', 'start']).sum().groupby(level=0).cumsum()
    df = df.set_index(['id', 'start'])
    df['cumur'] = cumsums
    df = df.reset_index()
    init = df[df['startglicko2rating'].notnull()].sort_values('start')[['startglicko2rating', 'id']].groupby('id').first().reset_index().rename(columns={'startglicko2rating': 'init'})
    df = pd.merge(df, init, how='left', on='id')
    df[ratingName] = df.cumur + df.init
    return df

def teamRating(rating, df):
    teameff = df.merge((df.groupby(['team', 'start']).mean()[rating] * 5).reset_index().rename(columns={rating: 'teamrating'}), on=['team', 'start'])
    teameff.teamrating = (teameff.teamrating - teameff[rating]) / 4
    return teameff

def experience(df):
    dfid2 = pd.DataFrame()
    for player in df.id.unique():
        dfid = pd.DataFrame(df[df['id'] == player].sort_values('start').tour.unique()).reset_index().rename(columns={'index': 'exp', 0: 'tour'})
        dfid['id'] = player
        dfid2 = pd.concat([dfid2, dfid])
    return pd.merge(df, dfid2, how='left', on=['id', 'tour'])
    
def dobMerge(df, dob):
    dob.id = dob.id.str.replace(r'\W+', '').str.lower()
    df = df.merge(dob, how='left', on=['id'])
    df.dob = pd.to_datetime(df.dob, format='%Y/%m/%d')
    df['age'] = (df.start - df.dob) / pd.Timedelta(days=365.25)
    return df

In [3]:
df2 = pd.read_csv('glickoandprize.csv', encoding="utf-8")
df2.start = pd.to_datetime(df2.start, format='%d/%m/%Y')
df2.end = pd.to_datetime(df2.end, format='%d/%m/%Y')
df2['diffr'] = df2.endglicko2rating - df2.startglicko2rating
df2 = cumulativeRating('diffr', 'normalRating', df2)
df2 = teamRating('normalRating', df2)
df2 = experience(df2)
#df2.drop_duplicates(subset=['id', 'start'], inplace=True)
#df2.dropna(inplace=True)
df2.prizepool = pd.to_numeric(df2.prizepool, errors='coerce')
df2.prizeusd = pd.to_numeric(df2.prizeusd.str.replace(r'\D', ''), errors='coerce')

In [4]:
df2['season'] = 'ti3'
df2.loc[(df2.start > pd.to_datetime('2013-08-02')) & (df2.start <= pd.to_datetime('2014-07-08')), 'season'] = 'ti4'
df2.loc[(df2.start > pd.to_datetime('2014-07-08')) & (df2.start <= pd.to_datetime('2015-07-27')), 'season'] = 'ti5'
df2.loc[(df2.start > pd.to_datetime('2015-07-27')) & (df2.start <= pd.to_datetime('2016-08-02')), 'season'] = 'ti6'
df2.loc[(df2.start > pd.to_datetime('2016-08-02')) & (df2.start <= pd.to_datetime('2017-08-02')), 'season'] = 'ti7'

df2['tiprize'] = np.log(int(df2[df2['tour'] == 'The International 2013'].prizepool.iloc[0]))
df2.loc[(df2.season == 'ti4'), 'tiprize'] = np.log(int(df2[df2['tour'] == 'The International 2014'].prizepool.iloc[0]))
df2.loc[(df2.season == 'ti5'), 'tiprize'] = np.log(int(df2[df2['tour'] == 'The International 2015'].prizepool.iloc[0]))
df2.loc[(df2.season == 'ti6'), 'tiprize'] = np.log(int(df2[df2['tour'] == 'The International 2016'].prizepool.iloc[0]))
df2.loc[(df2.season == 'ti7'), 'tiprize'] = np.log(int(df2[df2['tour'] == 'The International 2017'].prizepool.iloc[0]))

df2['wdiffr'] = (df2.prizepool.apply(np.log) / df2.tiprize) * df2.diffr

#cumulative sum of wdiffr
cumsums = df2[['id', 'start', 'wdiffr']].groupby(['id', 'start']).sum().groupby(level=0).cumsum()
df2 = df2.set_index(['id', 'start'])
df2['cumuwr'] = cumsums
df2 = df2.reset_index()
df2['perf'] = df2.cumuwr + df2.init

In [5]:
grouped = df2.groupby(['team', 'tour'])

In [6]:
pf = pd.DataFrame(columns=['tour', 'team', 'prizepool', 'prizeusd', 'perf1', 'perf2', 'perf3', 'perf4', 'perf5'])
for name, group in grouped:
    try:
        perf1 = group[group.pos == 1].perf.iloc[0]
    except:
        perf1 = np.nan
    try:
        perf2 = group[group.pos == 2].perf.iloc[0]
    except:
        perf2 = np.nan
    try:
        perf3 = group[group.pos == 3].perf.iloc[0]
    except:
        perf3 = np.nan
    try:
        perf4 = group[group.pos == 4].perf.iloc[0]
    except:
        perf4 = np.nan
    try:
        perf5 = group[group.pos == 5].perf.iloc[0]
    except:
        perf5 = np.nan
    pf = pf.append({'season': group.season.iloc[0],
                    'tour': name[1],
                    'start': group.start.iloc[0],
                    'team': name[0],
                    'prizepool': group.prizepool.iloc[0],
                    'prizeusd': group.prizeusd.iloc[0],
                    'perf1': perf1,
                    'perf2': perf2,
                    'perf3': perf3,
                    'perf4': perf4,
                    'perf5': perf5 
                   }, ignore_index=True)

In [7]:
pf = pf.dropna()
pf.prizeusd = pd.to_numeric(pf.prizeusd)
pf = pf.drop_duplicates(['team', 'start'])

In [8]:
import statsmodels.api as sm
Y = pf[['prizeusd']]
X = pf[['perf1', 'perf2', 'perf3', 'perf4', 'perf5']]
model = sm.OLS(Y,X)
results = model.fit()
results.summary()

C:\Users\terth\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prizeusd   R-squared:                  -10544.022
Model:                            OLS   Adj. R-squared:             -10583.844
Method:                 Least Squares   F-statistic:                    -264.8
Date:                Sat, 24 Mar 2018   Prob (F-statistic):               1.00
Time:                        00:13:31   Log-Likelihood:                -31071.
No. Observations:                1329   AIC:                         6.215e+04
Df Residuals:                    1324   BIC:                         6.218e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
perf1       -9.29e+05   6.24e+05     -1.489      0.137   -2.15e+06    2.95e+05
perf2       1.379e+06   7.19e+05      1.918      0.055   -3.14e+04    2.79e+06
perf3      -7.849e+05   7.42e+05     -1.058      0.290   -2.24e+06    6.71e+05
perf4      -1.155e+06   6.96e+05     -1.659      0.097   -2.52e+06    2.11e+05
perf5        1.61e+06   7.01e+05      2.298      0.022    2.36e+05    2.98e+06
==============================================================================
Omnibus:                     3102.229   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         18529549.381
Skew:                          21.842   Prob(JB):                         0.00
Kurtosis:                     579.811   Cond. No.                         37.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
ent = pd.DataFrame(pf.team.unique()).reset_index().rename(columns={'index': 'entity', 0: 'team'})
pf = ent.merge(pf, on='team', how='left')
pf.head()

,entity,team,tour,prizepool,prizeusd,perf1,perf2,perf3,perf4,perf5,season,start
0,0,3dmax,StarLadder StarSeries Season 5,15000.000000,0,1333.138496,1333.138496,1333.138496,1333.138496,1333.138496,ti3,2013-02-11
1,1,4friendschrillee,Corsair Gaming Summer Tournament 2013,10000.000000,0,1431.514235,1415.703233,1391.409425,1391.409425,1391.409425,ti3,2013-06-28
2,1,4friendschrillee,DreamHack Summer 2013,46422.000000,0,1427.237319,1427.237319,1427.237319,1427.237319,1427.237319,ti3,2013-06-15
3,1,4friendschrillee,Electronic Sports World Cup 2013,25000.000000,0,1449.456046,1439.095493,1449.456046,1449.456046,1479.450199,ti4,2013-10-31
4,1,4friendschrillee,Esportal Dota 2 League Invitational Tournament 1,18691.472328,623,1426.782769,1481.501966,1426.782769,1497.320470,1497.320470,ti5,2014-12-10


In [11]:
from linearmodels import PanelOLS 
panel_data = pf.set_index(['entity', 'start']).sort_index()
Y = np.add(1, panel_data['prizeusd']).apply(np.log)
X = panel_data[['perf1', 'perf2', 'perf3', 'perf4', 'perf5']].apply(np.log)
mod = PanelOLS(Y, X, time_effects=True)
mod.fit()

Dep. Variable:,prizeusd,R-squared:,0.1361
Estimator:,PanelOLS,R-squared (Between):,-348.47
No. Observations:,1329,R-squared (Within):,0.0508
Date:,"Sat, Mar 24 2018",R-squared (Overall):,-196.54
Time:,00:15:17,Log-likelihood,-3538.1
Cov. Estimator:,Unadjusted,,
,,F-statistic:,34.951
Entities:,103,P-value,0.0000
Avg Obs:,12.903,Distribution:,"F(5,1109)"
Min Obs:,1.0000,,
Max Obs:,77.000,F-statistic (robust):,34.951
